<a href="https://colab.research.google.com/github/Ngabo-bajo/NLP-FELLOWSHIP/blob/main/Web_scrapping_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing required libralies
import requests
from bs4 import BeautifulSoup as bs
import re 
import pandas as pd
import os

In [ ]:
# Mounting my google drive to google colab
from google.colab import drive
drive.mount('/content/gdrive')
base_path = "/content/gdrive/MyDrive/NLP-FELLOWSHIP/Week1/Data_Files"
os.chdir(base_path)

In [ ]:
# scrapping all snapshot links of igihe.com from wayback machine using API of wayback machine
snap_links = []
for month in range(10):
    for day in range(31):
        link = requests.get('http://archive.org/wayback/available?url=igihe.com&timestamp=2022{:02d}{:02d}'.format(month, day))
        try:
            if link.json()['archived_snapshots']['closest']['available']:
                snap_links.append(link.json()['archived_snapshots']['closest']['url'])
        except KeyError:
            pass

In [ ]:
# Removing all Duplicates in snaps_links without losing index position 
no_dupsLinks = []
ischecked = set()
for link in snap_links:
  if link not in ischecked:
    no_dupsLinks.append(link)
    ischecked.add(link)

In [ ]:
# Getting all article links and article headings from each snapshots
art_links = []
art_headings = []

for link in no_dupsLinks:
  Page_content = requests.get(link).text
  soup = bs(Page_content,'html.parser')
  articles = soup.find_all('span', class_='homenews-title')
  for item in articles:
    art_links.append(link+item.find('a')['href'])
    art_headings.append(item.find('a').text)

# Saving all links and Headings to CSV file

art = pd.DataFrame(list(zip(art_links,art_headings)),columns=['Article_Links','Article_Headings'])
art.to_csv('Igihe_ArticleLinks.csv')

In [ ]:
# scrapping all Articles from Jan to Oct and save each article to txt file
for i in range(len(art['Article_Links'])):
  Page_art = requests.get(art['Article_Links'][i]).content
  Page_soup = bs(Page_art)
  # creating Key name of each article according to dates puplished
  pattern = r'\d{8}'
  time = re.findall(pattern,art['Article_Links'][i])
  Key_name = time[0]
  wrap_artPage = Page_soup.find_all('div', class_='fulltext')
  article_content = ''
  for page in wrap_artPage:
    pageBody = page.find_all('p')
    for sent in pageBody:
      if sent.text:
        article_content += '\n'+ sent.text
    with open('./Igihe_articles/article_'+Key_name+f'-({i}).txt','w') as igihe:
      igihe.write(article_content)